In [ ]:
from fastcore.all import *
from fastai.vision.all import *
from fastprogress.fastprogress import force_console_behavior
from fastai.callback.progress import ProgressCallback
import fastai.callback.all
import fastprogress
import torch
import sys

# Force PyTorch to avoid MIOpen's problematic BatchNorm kernels
torch.backends.cudnn.enabled = False 

path = Path('bird_or_not')

dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=6)

In [ ]:
# Create your learner normally
learn = vision_learner(dls, resnet18, metrics=error_rate)
# Remove the problematic callback
learn.remove_cb(ProgressCallback)
learn.fine_tune(3)

In [ ]:
is_bird,_,probs = learn.predict(PILImage.create('bird.jpg'))
print(f"This is a: {is_bird}.")
print(f"Probability it's a bird: {probs[0]:.4f}")

In [ ]:
is_bird,_,probs = learn.predict(PILImage.create('forest.jpg'))
print(f"This is a: {is_bird}.")
print(f"Probability it's a bird: {probs[0]:.4f}")